In [11]:
from pyspark import SparkConf, SparkContext, SparkFiles
from pyspark.sql import SQLContext, SparkSession, Row

In [2]:
# spark session
spark = SparkSession.builder.appName("M5-CA2-ApacheLogs-TGA(with SQL)").getOrCreate() # singleton instance

In [12]:
location = "/user/edureka_672184/use_cases/Logs/access.clean.log"
raw = spark.read.text(location)
# type(raw) is dataframe
# r[0] is needed because each data is a Row object inside a list
df = raw.rdd.map(lambda r:r[0].split(" ")).map(lambda arr:Row(remote_host=arr[0], timestamp=arr[3].replace("[",""), request_type=arr[5],url=arr[6],status_code=arr[8])).toDF()
df.registerTempTable("logs")

In [15]:
spark.sql("SELECT count(*) FROM logs WHERE status_code='404'").show()

+--------+
|count(1)|
+--------+
|  227089|
+--------+



In [16]:
spark.sql("SELECT count(*) FROM logs WHERE status_code='204'").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [23]:
filterCond = " is Null or ".join(df.columns) + " is Null" 

spark.sql("SELECT COUNT(*) FROM logs WHERE " + filterCond).show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [24]:
spark.sql("select status_code, count(*) as cnt from logs group by status_code order by cnt desc").show()

+-----------+-------+
|status_code|    cnt|
+-----------+-------+
|        200|1157831|
|        206| 939929|
|        404| 227089|
|        304|   6330|
|        500|   3252|
|        403|   2222|
|        301|    619|
|        303|    247|
|        401|    153|
|        501|    143|
|        405|     83|
|        406|     53|
|        400|     23|
|        412|     19|
|   HTTP/1.1|     13|
+-----------+-------+



In [25]:
spark.sql("select remote_host, count(*) from logs group by remote_host").show()

+---------------+--------+
|    remote_host|count(1)|
+---------------+--------+
|  180.76.15.162|      75|
|  110.77.220.20|       1|
|  5.235.113.230|       1|
|  95.181.27.133|       1|
|180.153.232.170|       7|
|   199.30.24.91|      54|
|  91.121.79.180|     105|
| 162.253.64.155|       3|
|   62.210.80.77|       5|
|  196.16.15.228|       2|
|   83.94.47.160|       1|
|  95.108.181.71|      19|
|   196.17.9.145|       1|
|   89.35.30.210|       1|
| 147.75.210.233|      10|
|   91.141.0.218|      68|
|213.251.182.111|       8|
|213.177.106.194|       1|
| 217.70.253.127|       6|
| 177.73.233.241|       8|
+---------------+--------+
only showing top 20 rows

